In [ ]:
import numpy as np
import pandas as pd
pd.options.display.precision = 2
pd.options.display.max_columns = 50

In [ ]:
import re
import os
os.chdir('../../')

In [ ]:
from scipy.stats import pearsonr

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [15, 10]

In [ ]:
#import seaborn as sns
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Read Predictors data

In [ ]:
predictors_df = pd.read_csv('data/imf_cpi_indices_yearly.csv', index_col='year')

In [ ]:
original_predictors = predictors_df.columns
cols = original_predictors
predictors_df = predictors_df.join(predictors_df[cols].shift(1), rsuffix='_1y_shift')
predictors_df = predictors_df.join(predictors_df[cols].shift(2), rsuffix='_2y_shift')
predictors_df = predictors_df.loc[2003:2019]

In [ ]:
predictors_df

# Sponsor Report Dataset

## Read Dataset

In [ ]:
types = {
    'Student Id': str,
    'Availability year': str,
    'ETS availability year': str
}
data_df = pd.read_csv('data/sponsored_payments_raw.csv', dtype=types)
data_df.head()

## Clean up the data

### Remove entries with Status = 'Proposed'

In [ ]:
data_df.Status.value_counts()

In [ ]:
mask = data_df.Status == 'Proposed'
data_df[mask].describe().astype(int)

In [ ]:
data_df = data_df[~mask]

### Remove entries with "Availability Year" = *na

In [ ]:
year_column = 'Availability year'
na_value = '*na'
mask2 = data_df[year_column] == na_value
data_df[mask2].describe().astype(int)

In [ ]:
data_df[mask2].describe(include=['object']).T

In [ ]:
data_df = data_df[~mask2]
data_df[year_column] = data_df[year_column].astype(int)

## Dataset summary

In [ ]:
data_df.describe(include=['object']).T

In [ ]:
#data_df['Availability year'].value_counts()#.head(10)

In [ ]:
data_df.describe().astype(int).T

## Convert data into response format

In [ ]:
# configs for analysis
response_config = {
    'student_count':
    {
        'source_dataset': 
        {
            'column_name': 'Student Id',
            'agg_func': pd.Series.nunique
        },        
        'chart_properties':
        {
            'title': 'Number of Unique Sponsored Students',               
        }
    },
    'total_amount':
    {
        'source_dataset': 
        {
            'column_name': 'Actual Amount',
            'agg_func': sum
        },        
        'chart_properties':
        {
            'title': 'Total Paid Amount',               
        }
    }
}

factors_config = {
    'country': {
        'source_dataset':
        {
            'column_name': 'SPO country',
        },        
        'chart_properties':
        {
            'title': 'Country',               
        }
    },
    'sponsor_body':
    {
        'source_dataset':
        {
            'column_name': 'Sponsor Name',
        },        
        'chart_properties':
        {
            'title': 'Sponsor',
        }
    }
}
time_factor = 'year'
time_factor_config = {
    'title': 'Year'
}
factors_names = list(factors_config.keys())
LEVELS_TOP = 20
CONFIDENCE_LEVEL = 0.05

In [ ]:
# prepare dataframe with expected column names
col_names_map = {factors_config[key]['source_dataset']['column_name']: key for key in factors_config}
col_names_map[year_column] = time_factor
source_df = data_df.rename(columns=col_names_map)
#source_df.index.name = time_factor

In [ ]:
# aggregate raw data to get response data
temp_dfs = []

group_columns = [time_factor] + factors_names
for key in response_config.keys():
    config = response_config[key]['source_dataset']
    df = source_df.groupby(group_columns)[config['column_name']].agg(config['agg_func'])
    df.name = key
    temp_dfs.append(df)
    
response_df = pd.concat(temp_dfs, axis=1)

# Visualise

In [ ]:
def plot_bar_chart(df, y_config, x_config):
    traces = []
    traces.append(go.Bar(
            x = df.index.values,
            y = df.values,
            name = y_config['title'],
            opacity = 0.5))


    layout = dict(
        title = '{} By {}'.format(y_config['title'], x_config['title']),
        xaxis = dict(title = x_config['title']),

        yaxis = dict(
            title = y_config['title'],
            #range = [0, 250]
        ),

        legend=dict(
            x=0.3,
            y=1.1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            orientation="h"
        )
    ,
    )

    fig = dict(data=traces, layout=layout)
    iplot(fig)

In [ ]:
def plot_heatmap(correlation_df, response_var, factor_var):
    df = correlation_df.loc[correlation_df.index.values[::-1]]
    trace = go.Heatmap(z=df.values,
                       x=df.columns.values,
                       y=df.index.values,
                       zmin = -1,
                       zmax = 1
                      )
    adjust = lambda s: s.replace('_', ' ').title()
    response_var.capitalize()
    layout = go.Layout(
        title="Correlation Between {} and Commodity Indices by {}".format(adjust(response_var), 
                                                                              adjust(factor_var)),
        autosize=False,
        margin = dict(
                r = 10,
                t = 25,
                b = 200,
                l = 200),
        width=900,
        height=700)

    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)

In [ ]:
def plot_comparison_plot(resp_df, predictors_df, predictors, x_config, y_config):
    traces = []

    # Adding predictor lines
    for predictor in predictors:
        traces.append(
            go.Scatter(
                x=predictors_df.index.values,
                y=predictors_df[predictor].values,
                name=predictor,
                mode='lines'
            ),
        )

    # Adding response variable bar-chart by factor level
    factor_levels = resp_df.columns
    visibility = True
    for level in factor_levels:
        x = resp_df.index.values
        traces.append(go.Bar(
            x=x,
            y=resp_df[level],
            yaxis='y2',
            visible=visibility,
            name=y_config['title'],
            opacity = 0.5))
        visibility = False


    # Adding dropdown menu    
    buttons = []
    pr_count = len(predictors)
    for idx, level in enumerate(factor_levels):
        mask = [True] * pr_count + [False] * len(factor_levels)
        mask[pr_count + idx] = True
        buttons.append(
            dict(
                label = level,
                method = 'update',
                args = [{'visible': mask}]
            )
        )


    layout = dict(
        title = 'Trends in Commodity Prices and Number of Sponsored Students by Country',
        xaxis = dict(title = 'Year'),

        yaxis = dict(
            title = 'Commodity Index Value',
            range = [0, 250]
        ),

        yaxis2 = dict(
            title=y_config['title'],
            overlaying='y',
            side='right',
            range=[0, df.max().max() + 200]
            #position=0.15
        ),
        updatemenus = list([dict(
                active=0,
                pad = {'r': 0, 't': 0},
                x = 0.0,
                xanchor = 'left',
                y = 1.1,
                yanchor = 'top',
                buttons=buttons)]),
        legend=dict(
            x=0.3,
            y=1.1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            orientation="h"
        ),
        margin = dict(
                r = 50,
                t = 100,
                b = 50,
                l = 50)
    ,
    )


    fig = dict(data=traces, layout=layout)

    iplot(fig)

In [ ]:
def aggregate_by_factor(response_df, response_var, time_factor, factor_var, levels_top=20):
    group_columns = [time_factor, factor_var]
    resp_by_factor_df = response_df.groupby(level=group_columns)[response_var].agg(sum).unstack(factor_var, fill_value=0)
    top_levels = resp_by_factor_df.sum(axis=0).sort_values(ascending=False).index.values
    top_levels = top_levels[:min(top_levels.shape[0], levels_top)]
    return resp_by_factor_df[top_levels]

## Number of Sponsored Students

In [ ]:
response_var = list(response_config.keys())[0]

In [ ]:
df = response_df[response_var].groupby(level=time_factor).agg(sum)
y_config = response_config[response_var]['chart_properties']
plot_bar_chart(df, y_config, time_factor_config)

In [ ]:
for factor_var in list(factors_config.keys()):
    df = response_df[response_var].groupby(level=factor_var).agg(sum).sort_values(ascending=False).head(15)
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    plot_bar_chart(df, y_config, x_config)

### Correlation

In [ ]:
time_factor_values = predictors_df.index.values

for factor_var in list(factors_config.keys()):
    # transform response df to get data for particular factor
    resp_by_factor_df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    resp_by_factor_df = resp_by_factor_df.loc[time_factor_values]

    # Calculate correlation and pvalues
    factor_levels = resp_by_factor_df.columns
    predictors = predictors_df.columns
    correlations = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    pvalues = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    for idx, level in enumerate(factor_levels):
        for jdx, predictor in enumerate(predictors):
            correlation, pvalue = pearsonr(resp_by_factor_df[level], predictors_df[predictor])
            correlations[idx, jdx] = correlation
            pvalues[idx, jdx] = pvalue

    # Form dataframe with correlations
    correlation_df = pd.DataFrame(data=correlations, index=factor_levels, columns=predictors)
    pvalues_df = pd.DataFrame(data=pvalues, index=factor_levels, columns=predictors)
    significance_mask = pvalues_df < CONFIDENCE_LEVEL
    correlation_df[~significance_mask] = np.nan


    plot_heatmap(correlation_df, response_var, factor_var)

### Comparison Chart

In [ ]:
predictors = original_predictors[:5]
for factor_var in list(factors_config.keys()):
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    plot_comparison_plot(df, predictors_df, predictors, x_config, y_config)


## Total Paid Amount

In [ ]:
response_var = list(response_config.keys())[1]

In [ ]:
df = response_df[response_var].groupby(level=time_factor).agg(sum)
y_config = response_config[response_var]['chart_properties']
plot_bar_chart(df, y_config, time_factor_config)

In [ ]:
for factor_var in list(factors_config.keys()):
    df = response_df[response_var].groupby(level=factor_var).agg(sum).sort_values(ascending=False).head(15)
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    plot_bar_chart(df, y_config, x_config)

### Correlation

In [ ]:
time_factor_values = predictors_df.index.values

for factor_var in list(factors_config.keys()):
    # transform response df to get data for particular factor
    resp_by_factor_df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    resp_by_factor_df = resp_by_factor_df.loc[time_factor_values]

    # Calculate correlation and pvalues
    factor_levels = resp_by_factor_df.columns
    predictors = predictors_df.columns
    correlations = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    pvalues = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    for idx, level in enumerate(factor_levels):
        for jdx, predictor in enumerate(predictors):
            correlation, pvalue = pearsonr(resp_by_factor_df[level], predictors_df[predictor])
            correlations[idx, jdx] = correlation
            pvalues[idx, jdx] = pvalue

    # Form dataframe with correlations
    correlation_df = pd.DataFrame(data=correlations, index=factor_levels, columns=predictors)
    pvalues_df = pd.DataFrame(data=pvalues, index=factor_levels, columns=predictors)
    significance_mask = pvalues_df < CONFIDENCE_LEVEL
    correlation_df[~significance_mask] = np.nan


    plot_heatmap(correlation_df, response_var, factor_var)

### Comparison Chart

In [ ]:
predictors = original_predictors[:5]
for factor_var in list(factors_config.keys()):
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    plot_comparison_plot(df, predictors_df, predictors, x_config, y_config)
